Installation of requirements

In [1]:
!ls
!pip3 install torch torchvision
!pip3 install matplotlib
!pip install pyyaml==5.1
!pip3 install matplotlib
!pip3 install pillow
#!unzip ./drive/MyDrive/deploy.zip

'ls' is not recognized as an internal or external command,
operable program or batch file.


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pyyaml: filename=PyYAML-5.1-cp38-cp38-win_amd64.whl size=44070 sha256=5a4786a604845d3f6e180a66e850ddde8d00d43d2e3df59536ffe23ac84f511e
  Stored in directory: c:\users\alex\appdata\local\pip\cache\wheels\52\dd\2b\10ff8b0ac81b93946bb5fb9e6749bae2dac246506c8774e6cf
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 5.4.1
    Uninstalling PyYAML-5.4.1:
      Successfully uninstalled PyYAML-5.4.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.


Dataloader for the GTA Dataset

In [2]:
deploy_dir = "."
model_save_dir = ".models/"

"""
    Dataloaders for training and testing.
    Import the function readTrImages and MyDataset object for creating the training and validation dataloaders.
    Important!! - Make sure the deploy folder is in the same directory as this loader file. 
    TODO: Make testing code.       
"""
import sys
import os
import struct
import numpy as np
import torch
from torchvision import transforms
from PIL import Image
import random
from numpy.lib.shape_base import split
import torch.utils.data as data
from torch.utils.data import DataLoader, Dataset
from matplotlib.pyplot import imshow
from matplotlib import cm
import torch.nn.functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

classes = (
    'Unknown', 'Compacts', 'Sedans', 'SUVs', 'Coupes',
    'Muscle', 'SportsClassics', 'Sports', 'Super', 'Motorcycles',
    'OffRoad', 'Industrial', 'Utility', 'Vans', 'Cycles',
    'Boats', 'Helicopters', 'Planes', 'Service', 'Emergency',
    'Military', 'Commercial', 'Trains'
)
classes_to_labels = {
    'Unknown':0, 'Compacts':1, 'Sedans':1, 'SUVs':1, 'Coupes':1,
    'Muscle':1, 'SportsClassics':1, 'Sports':1, 'Super':1, 'Motorcycles':2,
    'OffRoad':2, 'Industrial':2, 'Utility':2, 'Vans':2, 'Cycles':2,
    'Boats':0, 'Helicopters':0, 'Planes':0, 'Service':0, 'Emergency':0,
    'Military':0, 'Commercial':0, 'Trains':0
}

class MyDataset(Dataset):
    # An object for representing the Dataset for Pytorch.
    def __init__(self, image_fns, labels, dim):
        super().__init__()
        self.image_fns = image_fns
        self.image_orig_shape = None
        self.labels = labels
        if not dim:
            self.dim = False
        else:
            self.dim = dim

    def __len__(self):
        return max(len(self.image_fns), len(self.labels))

    def getImage(self, index):
        image = Image.open(self.image_fns[index])
        self.image_orig_shape = image.size
        convert_tensor = transforms.ToTensor()
        image = torch.tensor(convert_tensor(image), requires_grad=True, device=device)
        image = F.interpolate(image, size=self.dim) 
        return image
    
    def getMask(self, index):
        #cols,rows = self.image_orig_shape
        bb = self.labels[index][0][1]
        label = self.labels[index][0][0][0]
        label = classes_to_labels[label]
        #Y = np.zeros((rows, cols))
        bb = np.array(bb)
        bb = bb.astype(np.int)
        #Y[bb[0]:bb[1], bb[1]:bb[2]] = 1.
        bb = torch.tensor([bb[2], bb[0], bb[3], bb[1]], dtype=torch.long, device=device)
        #mask = Image.fromarray(np.uint8(cm.gist_earth(Y)*255))
        #if self.dim:
            #mask = mask.resize((self.dim, self.dim))
        convert_tensor = transforms.ToTensor()
        #mask = torch.tensor(convert_tensor(mask), requires_grad=True, device=device)
        label = torch.tensor(label, dtype=torch.long, device=device)
        label = F.one_hot(label, num_classes=3)
        return label, bb

    def __getitem__(self, index):
        image = self.getImage(index)
        label, bb = self.getMask(index)
        target = {}
        target["boxes"] = bb
        target["labels"] = label
        #target["masks"] = mask
        return image, target

def writeFirstSubmission():
    # The code for the first submission.
    output = open('Team19.txt', 'w')
    output.write('guid/image,label\n')
    count = 0

    for folder in os.listdir(deploy_dir + '/deploy/test/'):
        for file in os.listdir(deploy_dir + '/deploy/test/' + folder):
            if file.endswith("_image.jpg"):
                output.write(folder + "/" + file[:-10] + ",1\n")
                count += 1

    print(count)
    output.close()
    
def readTrLabels(train=False):
    # Reading the training labels from the generated trainval_labels.csv file, which is generated by extract_info.py from Canvas.
    file_to_read = deploy_dir + "deploy/trainval/trainval_labels.csv"
    label_dict = {}
    file_data = open(file_to_read)
    count = 0
    for row in file_data:
        if count != 0:
            split_lst = row.split(',')
            label_dict[split_lst[0]] = int(split_lst[1][0])
        count += 1
    return label_dict

def rot(n):
    n = np.asarray(n).flatten()
    assert(n.size == 3)

    theta = np.linalg.norm(n)
    if theta:
        n /= theta
        K = np.array([[0, -n[2], n[1]], [n[2], 0, -n[0]], [-n[1], n[0], 0]])

        return np.identity(3) + np.sin(theta) * K + (1 - np.cos(theta)) * K @ K
    else:
        return np.identity(3)

def get_bbox(p0, p1):
    """
    Input:
    *   p0, p1
        (3)
        Corners of a bounding box represented in the body frame.

    Output:
    *   v
        (3, 8)
        Vertices of the bounding box represented in the body frame.
    *   e
        (2, 14)
        Edges of the bounding box. The first 2 edges indicate the `front` side
        of the box.
    """
    v = np.array([
        [p0[0], p0[0], p0[0], p0[0], p1[0], p1[0], p1[0], p1[0]],
        [p0[1], p0[1], p1[1], p1[1], p0[1], p0[1], p1[1], p1[1]],
        [p0[2], p1[2], p0[2], p1[2], p0[2], p1[2], p0[2], p1[2]]
    ])
    e = np.array([
        [2, 3, 0, 0, 3, 3, 0, 1, 2, 3, 4, 4, 7, 7],
        [7, 6, 1, 2, 1, 2, 4, 5, 6, 7, 5, 6, 5, 6]
    ], dtype=np.uint8)

    return v, e

def readBbox(snapshot):
    xyz = np.fromfile(snapshot.replace('_image.jpg', '_cloud.bin'), dtype=np.float32)
    xyz = xyz.reshape([3, -1])

    proj = np.fromfile(snapshot.replace('_image.jpg', '_proj.bin'), dtype=np.float32)
    proj.resize([3, 4])

    try:
        bbox = np.fromfile(snapshot.replace('_image.jpg', '_bbox.bin'), dtype=np.float32)
    except FileNotFoundError:
        print('[*] bbox not found.')
        bbox = np.array([], dtype=np.float32)
    
    bbox = bbox.reshape([-1, 11])

    uv = proj @ np.vstack([xyz, np.ones_like(xyz[0, :])])
    uv = uv / uv[2, :]

    bbox_list = []
    
    for k, b in enumerate(bbox):
        R = rot(b[0:3])
        t = b[3:6]

        sz = b[6:9]
        vert_3D, edges = get_bbox(-sz / 2, sz / 2)
        vert_3D = R @ vert_3D + t[:, np.newaxis]

        vert_2D = proj @ np.vstack([vert_3D, np.ones(vert_3D.shape[1])])
        vert_2D = vert_2D / vert_2D[2, :]
        min_x, min_y = 10000, 10000
        max_x, max_y = -1000, -1000

        for e in edges.T:
            #print(vert_2D[0, e], vert_2D[1, e]) # 2D bbox
            x1 = vert_2D[0, e][0]
            x2 = vert_2D[0, e][1]
            y1 = vert_2D[1, e][0]
            y2 = vert_2D[1, e][1]

            if y1 < min_y and y1 > 0:
                min_y = int(y1)
            if y1 > max_y:
                max_y = int(y1)
            if y2 < min_y and y2 > 0:
                min_y = int(y2)
            if y2 > max_y:
                max_y = int(y2)

            if x1 < min_x and x1 > 0:
                min_x = int(x1)
            if x1 > max_x:
                max_x = int(x1)
            if x2 < min_x:
                min_x = int(x2)
            if x2 > max_x:
                max_x = int(x2)
                       
            #print(vert_3D[0, e], vert_3D[1, e], vert_3D[2, e]) # 3d bbox

        c = classes[int(b[9])]
        bbox_list.append([[c], [min_y, max_y, min_x, max_x]])
    
    return bbox_list

def createDataset(images, labels, batch_size, dim):
    # Making dataset and dataloaders for PyTorch from a set of image filenames and labels.
    dataset = MyDataset(images ,labels, dim)
    loader = DataLoader(dataset,batch_size=batch_size, shuffle=False)
    return dataset, loader

def readTrImages(batch_size, split_ratio, dim=256, shuffle=False, train=False):
    # Reading and splitting the images from trainval and returning the dataset objects and the dataloaders for training.
    #labels_dict = readTrLabels()
    images = []
    labels = []
    deploy_trainval_dir = deploy_dir + '/deploy/trainval/'

    for folder in os.listdir(deploy_trainval_dir):
        if folder.endswith(".csv"):
            continue
        for file in os.listdir(deploy_trainval_dir + folder):
            if file.endswith("_image.jpg"):
                image_id = folder + "/" + file[:-10]
                #tmp_label = labels_dict[image_id]
                tmp_fn = deploy_trainval_dir + folder + "/" + file
                images.append(tmp_fn)
                #labels.append(tmp_label)
                bbox_list = readBbox(tmp_fn)
                labels.append(bbox_list)
                                             

    print(str(len(images)) + " images read.")
    print(str(len(labels)) + " labels read.")
    idx_shuffled = list(range(0, len(images)))

    if shuffle:
        random.shuffle(idx_shuffled)

    split_idx = int(split_ratio * len(images))
    train_images = [images[i] for i in idx_shuffled[:split_idx]]
    train_labels = [labels[i] for i in idx_shuffled[:split_idx]]
    val_images = [images[i] for i in idx_shuffled[split_idx:]]
    val_labels = [labels[i] for i in idx_shuffled[split_idx:]]

    train_dataset, train_loader = createDataset(train_images, train_labels, batch_size, dim)

    val_dataset, val_loader = createDataset(val_images, val_labels, batch_size, dim)
    print("Dataloaders created, train has " + str(len(train_dataset)) + " samples and val has " + str(len(val_dataset)) + " samples.")
    return train_loader, val_loader

Model Training Code:

In [3]:
# import the necessary packages
import sys
import torch
from torch import nn
import torchvision
import torch.nn.functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

from torch.nn import Dropout
from torch.nn import Identity
from torch.nn import Linear
from torch.nn import Module
from torch.nn import ReLU
from torch.nn import Sequential
from torch.nn import Sigmoid

from torch.nn import CrossEntropyLoss
from torch.nn import MSELoss
from torch.optim import Adam
from torchvision.models import resnet50

class Model(Module):
	def __init__(self, baseModel, numClasses, in_features=2048):
		super(Model, self).__init__()
		# initialize the base model and the number of classes
		self.baseModel = resnet50(pretrained=True)
		self.numClasses = numClasses
    # build the regressor head for outputting the bounding box
		# coordinates
		self.regressor = Sequential(
			Linear(in_features, 128),
			ReLU(),
			Linear(128, 64),
			ReLU(),
			Linear(64, 32),
			ReLU(),
			Linear(32, 4),
			Sigmoid()
		)
    # build the classifier head to predict the class labels
		self.classifier = Sequential(
			Linear(in_features, 512),
			ReLU(),
			Dropout(),
			Linear(512, 512),
			ReLU(),
			Dropout(),
			Linear(512, self.numClasses)
		)
		# set the classifier of our base model to produce outputs
		# from the last convolution block
		self.baseModel.fc = Identity()

	def forward(self, x):
		# pass the inputs through the base model and then obtain
		# predictions from two different branches of the network
		features = self.baseModel(x)
		bboxes = self.regressor(features)
		classLogits = self.classifier(features)
		classLogits = F.softmax(classLogits, dim=1)
		# return the outputs as a tuple
		return (bboxes, classLogits)
  
from torchsummary import summary
resnet = resnet50(pretrained=True)
model = Model(resnet50(pretrained=True), 3, 2048)
model = model.to(device=device)
summary(model, (3, 256, 256))



Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\Alex/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:02<00:00, 39.2MB/s]


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           9,408
       BatchNorm2d-2         [-1, 64, 128, 128]             128
              ReLU-3         [-1, 64, 128, 128]               0
         MaxPool2d-4           [-1, 64, 64, 64]               0
            Conv2d-5           [-1, 64, 64, 64]           4,096
       BatchNorm2d-6           [-1, 64, 64, 64]             128
              ReLU-7           [-1, 64, 64, 64]               0
            Conv2d-8           [-1, 64, 64, 64]          36,864
       BatchNorm2d-9           [-1, 64, 64, 64]             128
             ReLU-10           [-1, 64, 64, 64]               0
           Conv2d-11          [-1, 256, 64, 64]          16,384
      BatchNorm2d-12          [-1, 256, 64, 64]             512
           Conv2d-13          [-1, 256, 64, 64]          16,384
      BatchNorm2d-14          [-1, 256,

In [4]:
resnet = resnet50(pretrained=True)
# freeze all ResNet50 layers so they will *not* be updated during the
# training process
for param in resnet.parameters():
    param.requires_grad = False

# create our custom object detector model and flash it to the current
# device
objectDetector = Model(resnet50, 3)
objectDetector = objectDetector.to(device=device)
# define our loss functions
classLossFunc = CrossEntropyLoss()
bboxLossFunc = MSELoss()
# initialize the optimizer, compile the model, and show the model
# summary
# print(objectDetector)
# initialize a dictionary to store training history
H = {"total_train_loss": [], "total_val_loss": [], "train_class_acc": [],"val_class_acc": []}
  

def train(model, train_loader, val_loader, num_epochs):
    classLossFunc = torch.nn.BCELoss()
    bboxLossFunc = MSELoss()
    lr = 1e-4
    optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
    start_epoch = 0
    
    num_iters = len(train_loader)

    for epoch in range(start_epoch, num_epochs):  # loop over the dataset multiple times  
        print("Training epoch: " + str(epoch) + " of " + str(num_epochs))
        totalTrainLoss = 0
        totalValLoss = 0
        # initialize the number of correct predictions in the training
        # and validation step
        trainCorrect = 0
        valCorrect = 0
        model.train()
        for i, data in enumerate(train_loader):
            # get the inputs; data is a list of [batch, labels]
            print("Started Iteration: " + str(i) + " of " + str(num_iters))
            batch, targets = data
            bboxes = targets["boxes"]
            labels = targets["labels"]
            bboxes = bboxes.type(torch.float)
            labels = labels.type(torch.float)
            predictions = model(batch)
            bboxLoss = bboxLossFunc(predictions[0], bboxes)
            classLoss = classLossFunc(predictions[1], labels.float())
            totalLoss = (1 * bboxLoss) + (1 * classLoss)
            optimizer.zero_grad()
            totalLoss.backward()
            print("Backward done on this iteration, learning rate - ", lr)
            optimizer.step()
            print("Model finished running iteration", i, "\n")
            print("Loss:", totalLoss)
            print("Running Loss:", totalTrainLoss)
            totalTrainLoss += totalLoss
            trainCorrect += (predictions[1].argmax(1) == labels.argmax(1)).type(torch.float).sum().item()

        with torch.no_grad():
            # set the model in evaluation mode
            model.eval()
            # loop over the validation set
            for i, data in enumerate(val_loader):
              # get the inputs; data is a list of [batch, labels]
              print("Started Iteration: " + str(i) + " of " + str(num_iters))
              batch, targets = data
              bboxes = targets["boxes"]
              labels = targets["labels"]
              bboxes = bboxes.type(torch.float)
              labels = labels.type(torch.float)
              predictions = model(batch)
              bboxLoss = bboxLossFunc(predictions[0], bboxes)
              classLoss = classLossFunc(predictions[1], labels.float())
              totalLoss = (1 * bboxLoss) + (1 * classLoss)
              totalValLoss += totalLoss

              # calculate the number of correct predictions
              valCorrect += (predictions[1].argmax(1) == labels).type(torch.float).sum().item()
        	# calculate the average training and validation loss

        avgTrainLoss = totalTrainLoss / trainSteps
        avgValLoss = totalValLoss / valSteps
        # calculate the training and validation accuracy
        trainCorrect = trainCorrect / len(train_loader)
        valCorrect = valCorrect / len(val_loader)
        # update our training history
        H["total_train_loss"].append(avgTrainLoss.cpu().detach().numpy())
        H["train_class_acc"].append(trainCorrect)
        H["total_val_loss"].append(avgValLoss.cpu().detach().numpy())
        H["val_class_acc"].append(valCorrect)
        # print the model training and validation information
        print("[INFO] EPOCH: {}/{}".format(epoch + 1, num_epochs))
        print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(avgTrainLoss, trainCorrect))
        print("Val loss: {:.6f}, Val accuracy: {:.4f}".format(avgValLoss, valCorrect))  

        
        model_fn = model_save_dir + "/model_epoch" + str(epoch) + "_loss" + str(avgTrainLoss) + "_vacc" + str(valCorrect) + ".pt"
        torch.save({
        'epoch': (epoch+1),
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': avgTrainLoss,
        'vacc': valCorrect
        }, model_fn)


    print('Finished Training')
    return model, H

train_loader, val_loader = readTrImages(4, 0.7, dim=400)
torch.cuda.empty_cache()
model = train(objectDetector, train_loader, val_loader, 20)

Model(
  (baseModel): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          

KeyboardInterrupt: 

Testing and output

In [ ]:
import os
import sys
import torch
from torchvision import transforms
from PIL import Image
from torch import nn

def runModel(model, img_fn, dim):
    image = Image.open(img_fn)
    image = image.resize((dim, dim))
    convert_tensor = transforms.ToTensor()
    x = convert_tensor(image)
    x = torch.stack([x])
    x = x.float()
    outputs = model(x)
    y = torch.argmax(outputs[1], dim=1).item()
    return y

def writeSubmission(model, dim=224):
    # The code for the submission after running the model.
    output = open(model_save_dir + '/Team19.txt', 'w')
    output.write('guid/image,label\n')
    count = 0

    for folder in os.listdir(deploy_dir +'/deploy/test/'):
        for file in os.listdir(deploy_dir + '/deploy/test/' + folder):
            if file.endswith("_image.jpg"):
                tmp_fn = deploy_dir + '/deploy/test/' + folder + "/" + file
                out_label = runModel(model, tmp_fn, dim)
                output.write(folder + "/" + file[:-10] + "," + str(out_label) + "\n")
                count += 1

    print(count)
    output.close()

